In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input
from tensorflow.keras.layers import Dense,Conv2D,GlobalAvgPool2D,Input
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras import callbacks,optimizers
import numpy as np
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/New Plant Diseases Dataset(Augmented)

/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)


In [ ]:
for i in os.listdir("train"):
   print(i,len(os.listdir("train/"+i)))


Tomato___healthy 1926
Tomato___Late_blight 1851
Tomato___Target_Spot 1827
Tomato___Leaf_Mold 1882
Tomato___Bacterial_spot 1702
Tomato___Early_blight 1920
Tomato___Spider_mites Two-spotted_spider_mite 1741
Tomato___Tomato_Yellow_Leaf_Curl_Virus 1961
Tomato___Tomato_mosaic_virus 1790
Tomato___Septoria_leaf_spot 1745


In [ ]:
for i in os.listdir("valid"):
   print(i,len(os.listdir("valid/"+i)))


Tomato___Early_blight 480
Tomato___Bacterial_spot 425
Tomato___Target_Spot 457
Tomato___Spider_mites Two-spotted_spider_mite 435
Tomato___Tomato_mosaic_virus 448
Tomato___Leaf_Mold 470
Tomato___healthy 481
Tomato___Septoria_leaf_spot 436
Tomato___Late_blight 463
Tomato___Tomato_Yellow_Leaf_Curl_Virus 490


In [ ]:
def img_Data(dir_path,target_size,batch,class_list,preprocessing,):
  if preprocessing:
    gen_object = ImageDataGenerator(preprocessing_function=preprocessing)
  else:
    gen_object = ImageDataGenerator()

  return(gen_object.flow_from_directory(dir_path,target_size,batch_size=batch,class_mode='sparse',classes=class_list,shuffle=True))

In [ ]:
train_data_gen = img_Data("train",(224,224),500,os.listdir("train"),preprocess_input)
valid_data_gen = img_Data("valid",(224,224),500,os.listdir("valid"),preprocess_input)

Found 18345 images belonging to 10 classes.
Found 4585 images belonging to 10 classes.


In [ ]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
             input_shape=(224,224,3), alpha=1.0, include_top=False, weights='imagenet',
             input_tensor=None, pooling=None, classes=1000,
             classifier_activation='softmax')

9420800/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.trainable = False

In [ ]:
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAvgPool2D())
model.add(Dense(1024,activation='relu'))
model.add(Dense(10,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
elst = callbacks.EarlyStopping(monitor='val_loss',patience=5,mode='min')
save_ck = callbacks.ModelCheckpoint('.mdl_wts.hdf5',save_best_only=True,monitor='val_loss',mode='min')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                 
 dense_1 (Dense)             (None, 10)                10250     
                                                                 
Total params: 3,579,978
Trainable params: 1,321,994
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
model.fit(train_data_gen,batch_size=500,validation_data=valid_data_gen,callbacks=[elst,save_ck],epochs=10)

Epoch 1/10
37/37 [==============================] - ETA: 0s - loss: 0.8708 - accuracy: 0.7181  

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


37/37 [==============================] - 5464s 148s/step - loss: 0.8708 - accuracy: 0.7181 - val_loss: 7.8735 - val_accuracy: 0.0979
Epoch 2/10
37/37 [==============================] - 91s 2s/step - loss: 0.3209 - accuracy: 0.8927 - val_loss: 8.9923 - val_accuracy: 0.0951
Epoch 3/10
37/37 [==============================] - 99s 3s/step - loss: 0.2380 - accuracy: 0.9225 - val_loss: 9.5676 - val_accuracy: 0.0912
Epoch 4/10
37/37 [==============================] - 98s 3s/step - loss: 0.1922 - accuracy: 0.9401 - val_loss: 10.3814 - val_accuracy: 0.0890
Epoch 5/10
37/37 [==============================] - 95s 3s/step - loss: 0.1537 - accuracy: 0.9549 - val_loss: 10.8588 - val_accuracy: 0.0870
Epoch 6/10
37/37 [==============================] - 95s 3s/step - loss: 0.1233 - accuracy: 0.9644 - val_loss: 11.4376 - val_accuracy: 0.0931
